
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [3]:
# the path to your HDF5 formatted snps file
data = "./G_cy_5rm_v9.snps.hdf5"

In [4]:
imap = {
#"ref": ["reference"],
"Inam": ["G_cy_T12385_In","G_cy_T310_In","G_cy_T23196_In","G_cy_T7636_In"],
"Puru": ["G_cy_82508_pu","G_cy_T12275_pu","G_cy_T12279_pu","G_cy_T12392_pu","G_cy_T13184_pu","G_cy_T26228_pu","G_cy_T26229_pu","G_cy_T26252_pu"],
"JiGu": ["G_cy_T3343_jigu","G_cy_T3384_jigu","G_cy_T3385_jigu"],
#"Mach": ["G_cy_J296_ma","G_cy_J477_ma","G_cy_J773_ma","G_cy_T13251_ma","G_cy_T363_ma","G_cy_T364_ma"],
#"Roar": ["G_cy_J691_roar","G_cy_J694_roar"],
#"ArSu": ["G_cy_80582_arsu","G_cy_85678_arsu","G_cy_80701_arsu","G_cy_80801_arsu","G_cy_80826_arsu","G_cy_81108_arsu","G_cy_81118_arsu","G_cy_85499_arsu"],
#"SuTa": ["G_cy_85356_suta","G_cy_86297_suta","G_cy_86321_suta","G_cy_86458_suta","G_cy_86478_suta","G_cy_T14558_suta","G_cy_T16693_suta","G_cy_T18563_suta","G_cy_T18620_suta"],
#"Para": ["G_cy_T10897_pa","G_cy_T11062_pa","G_cy_T16771_pa","G_cy_T1705_pa","G_cy_T18744_pa","G_cy_T19429_pa","G_cy_T19520_pa","G_cy_T19765_pa","G_cy_T2497_pa","G_cy_T6579_pa","G_cy_T9133_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="G_cy_str_west",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 15
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 8248
Filtered (mincov): 1104676
Filtered (minmap): 1104676
Filtered (combined): 1104917
Sites after filtering: 25620
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000
#struct.write_structure_files(abs)

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 2:31:47 | running 70 structure jobs 


In [8]:
struct = ipa.structure(
    data=data, 
    name="G_cy_str_west", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [G_cy_str_west]


In [10]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-8204.03,211.382,0.00,0.00
3,10,0.549,-8336.00,227.811,-131.97,125.16
4,10,4.073,-8593.13,900.557,-257.13,3668.20
5,10,0.558,-12518.46,10489.070,-3925.33,5850.45
6,10,1.804,-10593.34,2796.126,1925.12,5043.83
7,10,0.345,-13712.05,4738.060,-3118.71,1634.67
8,10,0.000,-18465.43,8180.655,-4753.38,0.00


In [11]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t593ba036feb545b78e0cd31039636a92" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 9000 12000 15000 18000 estLnProbMean 0 1 2 3 4 5 deltaK

In [12]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [13]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [14]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t38a285aa63bb4dbf82804b7dd70160c4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu 0.0 0.5 1.0

In [15]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [16]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [17]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t7ed1e855e3b44c2eb94c6efa8bc316b1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu 0.0 0.5 1.0

In [18]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [19]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [20]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tf7bdcd8d0f3c452a97d263b07e12b2d8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu 0.0 0.5 1.0

In [21]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [22]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [23]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t5dcdc740727849c29f049df9a308331e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu 0.0 0.5 1.0

In [24]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [25]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [26]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t044ec65e328240f89942dd03274ec170" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu 0.0 0.5 1.0

# PCA

In [ ]:
#!conda install scikit-learn -c conda-forge -y

In [5]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [6]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 15
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 8248
Filtered (mincov): 676521
Filtered (minmap): 1104676
Filtered (combined): 1104917
Sites after filtering: 25620
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [7]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 3474/25620


In [8]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2ab527e88950>,
 <toyplot.mark.Point at 0x2ab52c456d10>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t5d4a492e6de54910aec991b1ce1e35c7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12385_In G_cy_T12392_pu G_cy_T13184_pu G_cy_T23196_In G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T310_In G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_T7636_In -5 0 5 10 PC0 (25.0%) explained -8 -4 0 4 8 PC1 (10.5%) explained Inam Puru JiGu

In [9]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("G_cy_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
G_cy_82508_pu,-7.24,-6.40,-1.18,-5.21,-1.46,-0.78,0.40,0.90,0.12,0.55
G_cy_T12275_pu,-5.75,5.40,-0.28,2.94,-6.14,0.88,8.60,1.90,2.37,0.40
G_cy_T12279_pu,-6.13,5.82,-1.79,0.73,-0.12,-0.17,-3.12,7.10,-2.81,-4.91
G_cy_T12385_In,4.96,1.76,8.47,-3.67,-0.99,9.60,-1.73,0.52,-0.61,0.13
G_cy_T12392_pu,-6.91,7.83,-3.59,-1.78,-3.14,-0.90,-6.02,-6.72,-1.13,2.18
G_cy_T13184_pu,-6.04,3.73,-1.11,-0.88,11.31,1.05,2.49,0.47,2.97,3.66
G_cy_T23196_In,9.00,-1.35,-3.68,2.40,-2.06,1.32,1.95,0.30,-4.14,7.61
G_cy_T26228_pu,-5.12,-3.78,3.05,10.09,1.36,1.28,-1.69,-3.00,-0.33,-1.62
G_cy_T26229_pu,-6.91,-6.64,-1.13,-5.18,-1.51,-0.82,0.39,0.99,0.09,0.54
G_cy_T26252_pu,-6.16,-6.88,1.18,2.45,-0.22,0.35,-0.75,-1.14,-0.19,-0.77


# Subsampling with replication

In [10]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 2);

Subsampling SNPs: 3474/25620


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t94e84b0298194bfe814c19c8e5533a27" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12385_In G_cy_T12392_pu G_cy_T13184_pu G_cy_T23196_In G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T310_In G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_T7636_In -10 -5 0 5 10 PC0 (26.3%) explained -5 0 5 10 PC2 (8.2%) explained Inam Puru JiGu

# Kmeans imputation (integer)

In [11]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 2);

Subsampling SNPs: 3474/25620


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t088bebc3ad1f49f7864c539588adf861" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12385_In G_cy_T12392_pu G_cy_T13184_pu G_cy_T23196_In G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T310_In G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_T7636_In -10 -5 0 5 10 PC0 (26.5%) explained -5 0 5 10 15 PC2 (8.4%) explained Inam Puru JiGu

In [12]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "G_cy_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [13]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [27]:
pca.run_tsne(subsample=True, perplexity=3.0, n_iter=100000, seed=123)

Subsampling SNPs: 3474/25620


In [30]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="taae1e70d05c643d78b7ea8b8d60bbdb5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12385_In G_cy_T12392_pu G_cy_T13184_pu G_cy_T23196_In G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T310_In G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_T7636_In 0 100 200 TNSE component 1 -500 0 500 TNSE component 2 Inam Puru JiGu

In [16]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "G_cy_TSNE_perp8.pdf")

# GENETIC DISTNANCES

In [17]:
import ipyrad.analysis as ipa
import toyplot

In [18]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 15
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 8248
Filtered (mincov): 676521
Filtered (minmap): 1104676
Filtered (combined): 1104917
Sites after filtering: 25620
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


In [19]:
# save to a CSV file
dist.dists.to_csv("G_cy_distances.csv")

In [20]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "G_cy_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [21]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [22]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t5b8572dc1ef34dcb82b70f46727db9f6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.113856 0.115652 0.113739 0.125878 0.138954 0.135792 0.139383 0.138759 0.132865 0.129938 0.125059 0.115886 0.111632 0.112763 G_cy_T12385_In 1 0.113856 0.000000 0.126776 0.124083 0.123419 0.132904 0.132631 0.133411 0.134895 0.130484 0.127322 0.121975 0.120609 0.124395 0.121624 G_cy_T310_In 2 0.115652 0.126776 0.000000 0.095902 0.137237 0.148751 0.153005 0.153864 0.150195 0.149141 0.140749 0.138759 0.098361 0.095355 0.095394 G_cy_T23196_In 3 0.113739 0.124083 0.095902 0.000000 0.131109 0.143247 0.145863 0.148517 0.144067 0.143247 0.135480 0.132865 0.097073 0.097424 0.092077 G_cy_T7636_In 4 0.125878 0.123419 0.137237 0.131109 0.000000 0.094340 0.094067 0.092662 0.093208 0.093247 0.004762 0.041257 0.135909 0.135714 0.133802 G_cy_82508_pu 5 0.138954 0.132904 0.148751 0.143247 0.094340 0.000000 0.094340 0.093404 0.103864 0.103591 0.098244 0.098517 0.147736 0.149180 0.149219 G_cy_T12275_pu 6 0.135792 0.132631 0.153005 0.145863 0.094067 0.094340 0.000000 0.091725 0.100390 0.102537 0.098283 0.096526 0.147541 0.150156 0.144731 G_cy_T12279_pu 7 0.139383 0.133411 0.153864 0.148517 0.092662 0.093404 0.091725 0.000000 0.100625 0.105504 0.096956 0.096214 0.151600 0.149454 0.149258 G_cy_T12392_pu 8 0.138759 0.134895 0.150195 0.144067 0.093208 0.103864 0.100390 0.100625 0.000000 0.101991 0.096956 0.095667 0.149727 0.151171 0.149727 G_cy_T13184_pu 9 0.132865 0.130484 0.149141 0.143247 0.093247 0.103591 0.102537 0.105504 0.101991 0.000000 0.096916 0.051210 0.147034 0.148946 0.144770 G_cy_T26228_pu 10 0.129938 0.127322 0.140749 0.135480 0.004762 0.098244 0.098283 0.096956 0.096956 0.096916 0.000000 0.045628 0.139266 0.139774 0.137705 G_cy_T26229_pu 11 0.125059 0.121975 0.138759 0.132865 0.041257 0.098517 0.096526 0.096214 0.095667 0.051210 0.045628 0.000000 0.136105 0.138251 0.133685 G_cy_T26252_pu 12 0.115886 0.120609 0.098361 0.097073 0.135909 0.147736 0.147541 0.151600 0.149727 0.147034 0.139266 0.136105 0.000000 0.094653 0.094536 G_cy_T3343_jigu 13 0.111632 0.124395 0.095355 0.097424 0.135714 0.149180 0.150156 0.149454 0.151171 0.148946 0.139774 0.138251 0.094653 0.000000 0.092545 G_cy_T3384_jigu 14 0.112763 0.121624 0.095394 0.092077 0.133802 0.149219 0.144731 0.149258 0.149727 0.144770 0.137705 0.133685 0.094536 0.092545 0.000000 G_cy_T3385_jigu